In [21]:
from typing import Any
from py3.k_means import PixelValueGenerator, CustomKMeans
import numpy as np
from py3.CNN_model import Cplx_CustomCNN_1D
from py3.Attention_Layer import CustomAttentionLayer, AttentionLayer
from py3.classification import CustomClassifierModel
import tensorflow as tf

import re
import numpy as np
import pandas as pd

import pandas as pd
import numpy as np

In [22]:
# DATA
#_________________________________________________________________
# Load the data
df = pd.read_csv('Prepro_Data/post_processed_data_small20.csv')

def extract_multiple_arrays(string_repr):
    # Remove letters, parentheses, spaces, and unwanted sequences
    string_repr = re.sub('[a-df-zA-DF-Z\(\)\s]', '', string_repr)
    string_repr = re.sub(r',=32', '', string_repr)

    # Find all matches of arrays within the string
    arrays = re.findall(r'\[.*?\]', string_repr)

    # Convert each found array string into a NumPy array
    #np_arrays = [np.array(re.findall(r'[+-]?\d+(?:\.\d+)?', array), dtype=float) for array in arrays]
    np_arrays = [np.array(re.findall(r'[+-]?\d+(?:\.\d+)?(?:e[+-]?\d+)?', array), dtype=float) for array in arrays]
    return np_arrays

df['pixels'] = df['pixels'].apply(extract_multiple_arrays)
print(df.head())

   Unnamed: 0  segment_id  polygon_id  class_id  \
0           0           0           0         0   
1           1           1           0         0   
2           2           2           0         0   
3           3           3           0         0   
4           4           4           0         0   

                                              pixels  
0  [[0.47762346, 0.41975307, 0.36265433, 0.590277...  
1  [[0.6566358, 0.65123457, 0.6458333, 0.7507716,...  
2  [[0.51929015, 0.46682099, 0.41435185, 0.621141...  
3  [[0.525463, 0.49305555, 0.46141976, 0.6535494,...  
4  [[0.5146605, 0.49845678, 0.48225307, 0.6195987...  


In [23]:
print(df.shape)
""" j = 1
for i in (df['pixels'][30][10]):
    print(j)
    print(i)
    j+=1 """
    
# show unique values in df['pixels'][30][10]
print((np.unique(df['pixels'][0][0])))

(3543, 5)
[0.0625     0.07098766 0.07407408 0.07947531 0.08587651 0.09182098
 0.09413581 0.09581263 0.09722222 0.09985802 0.11071682 0.1121628
 0.12494084 0.13555713 0.1376863  0.15471967 0.1566493  0.16394606
 0.1718385  0.1736111  0.17955998 0.17983909 0.18364197 0.1991992
 0.19943222 0.20302887 0.20470588 0.20965452 0.2148604  0.2183642
 0.2191358  0.22222222 0.22489156 0.22782116 0.24074075 0.24325603
 0.24391058 0.25091758 0.25615504 0.25848764 0.25915965 0.26046842
 0.26117647 0.26234567 0.2640795  0.27608234 0.28034067 0.28395063
 0.28530872 0.28706127 0.28780866 0.28858024 0.28862196 0.2902768
 0.29320988 0.29524484 0.2955247  0.2978151  0.3013013  0.30241364
 0.3052532  0.30555555 0.3085385  0.31014904 0.31095678 0.31566492
 0.31598264 0.31635803 0.31796753 0.31798318 0.3186657  0.32008517
 0.32150462 0.3232371  0.32407406 0.32561728 0.32739654 0.32938948
 0.3294753  0.3337815  0.3347899  0.33647057 0.33873457 0.34137687
 0.3426408  0.3462546  0.34989354 0.35628104 0.35636535 

In [24]:
# ALGORITHM
#_________________________________________________________________
# K-means
stacked_arrays = []

# Stack the arrays for each cell
for i, cell_pixels in enumerate(df['pixels']):
    stacked_array = np.vstack(cell_pixels)
    stacked_arrays.append(stacked_array)
        
n_clusters=2
custom_kmeans = CustomKMeans(n_clusters=n_clusters)

clustered_data = []
clustered_labels = []
for i in range (len(stacked_arrays)):
    custom_kmeans.fit(stacked_arrays[i])
    clustered_data.append(custom_kmeans.get_cluster_centers())
    clustered_labels.append(custom_kmeans.get_cluster_labels())
    
clusters = np.array(clustered_data)
print(clusters.shape)

/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/si

(3543, 2, 73, 10)


/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/si

In [25]:
class Custom_Model(tf.keras.Model):
    '''
    This class defines the model architecture.
    '''
    
    def __init__(self):
        super(Custom_Model, self).__init__()
        self.enc = Cplx_CustomCNN_1D()
        self.attn = AttentionLayer()
        self.classifier = CustomClassifierModel(num_classes=8, fc_units=64) 

    def call(self, inputs):
        input_list = tf.unstack(inputs,axis=1) # from tensor to list
        intermediate = [self.enc(input) for input in input_list]
        intermediate = tf.stack(intermediate,axis=1)        
        emb, alphas = self.attn(intermediate)
        
        return self.classifier(emb)
    
    def get_alphas(self, inputs):
        input_list = tf.unstack(inputs,axis=1) # from tensor to list
        intermediate = [self.enc(input) for input in input_list]
        intermediate = tf.stack(intermediate,axis=1)        
        _, alphas = self.attn(intermediate)
        
        return alphas


In [26]:
model = Custom_Model()
input_shape = (2, 73, 10)  # Define your input shape

# Create a dummy input tensor for model initialization
dummy_input = tf.random.normal((1,) + input_shape)

# Pass the dummy input through the model to initialize the layers
model(dummy_input)


<tf.Tensor: shape=(1, 8), dtype=float32, numpy=
array([[0.12051178, 0.12422085, 0.12143738, 0.12244345, 0.12621194,
        0.12125302, 0.13444003, 0.12948158]], dtype=float32)>

In [27]:
# Load the weights
SAVE_PATH = 'saved_models/'
model.load_weights(SAVE_PATH + 'model_weights_100_epochs.h5')

In [36]:
import math

predictions = model.predict(clusters)

df_inferring = df.iloc[:, :4]
# Get the class with the highest probability
df_inferring['class_id'] = np.argmax(predictions, axis=1) + 1
df_inferring['class_id'] = df_inferring['class_id'].astype('int8')

# Pixels' cluster
# Assuming you have 'clustered_labels' available
df_inferring['pixels_cluster'] = clustered_labels

# Alphas
alphas = model.get_alphas(clusters)
alphas = np.array(alphas)
df_inferring['alphas'] = alphas.tolist()

# Function to round to the nearest hundredth
def round_to_hundredth(lst):
    return [math.ceil(num * 100) / 100 for num in lst]

# Apply the rounding function to the 'alphas' column
df_inferring['alphas'] = df_inferring['alphas'].apply(round_to_hundredth)


print(df_inferring.head())

# Save the results
df_inferring.to_csv('Inference_results.csv', index=False)

111/111 [==============================] - 2s 19ms/step
   Unnamed: 0  segment_id  polygon_id  class_id  \
0           0           0           0         6   
1           1           1           0         6   
2           2           2           0         6   
3           3           3           0         5   
4           4           4           0         5   

                                      pixels_cluster        alphas  
0  [1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...  [0.34, 0.67]  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...  [0.26, 0.75]  
2  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  [0.64, 0.37]  
3  [1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, ...  [0.46, 0.55]  
4  [1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, ...  [0.65, 0.36]  


In [37]:
# import csv, make it a df
df_pixels = pd.read_csv('Prepro_Data/filtered_segments_20_ordered (1).txt')
df_pixels = pd.DataFrame(df_pixels)
df_pixels.head()

,Segment ID,Polygon ID,Class ID,Pixels,Perimeter Pixels
0,0,0,0,"[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0...","[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 4..."
1,1,0,0,"[(0, 5), (0, 6), (0, 7), (0, 8), (1, 5), (1, 6...","[(0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (4, 8..."
2,2,0,0,"[(0, 9), (0, 10), (0, 11), (0, 12), (1, 9), (1...","[(0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (4, 1..."
3,3,0,0,"[(0, 13), (0, 14), (0, 15), (0, 16), (1, 13), ...","[(0, 13), (1, 13), (2, 13), (3, 13), (4, 13), ..."
4,4,0,0,"[(0, 17), (0, 18), (0, 19), (1, 16), (1, 17), ...","[(0, 17), (1, 16), (2, 16), (3, 16), (4, 17), ..."


In [38]:
# add df_pixels[Pixels] at 5th and df_inferring['Perimeter Pixels'] at 6th column of df_inferring
df_inferring.insert(5, "Pixels", df_pixels['Pixels'])
df_inferring.insert(6, "Perimeter Pixels", df_pixels['Perimeter Pixels'])
# delete df_inferring[pixels] column
df_inferring.head()

,Unnamed: 0,segment_id,polygon_id,class_id,pixels_cluster,Pixels,Perimeter Pixels,alphas
0,0,0,0,6,"[1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...","[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0...","[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 4...","[0.34, 0.67]"
1,1,1,0,6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[(0, 5), (0, 6), (0, 7), (0, 8), (1, 5), (1, 6...","[(0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (4, 8...","[0.26, 0.75]"
2,2,2,0,6,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(0, 9), (0, 10), (0, 11), (0, 12), (1, 9), (1...","[(0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (4, 1...","[0.64, 0.37]"
3,3,3,0,5,"[1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, ...","[(0, 13), (0, 14), (0, 15), (0, 16), (1, 13), ...","[(0, 13), (1, 13), (2, 13), (3, 13), (4, 13), ...","[0.46, 0.55]"
4,4,4,0,5,"[1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, ...","[(0, 17), (0, 18), (0, 19), (1, 16), (1, 17), ...","[(0, 17), (1, 16), (2, 16), (3, 16), (4, 17), ...","[0.65, 0.36]"


In [39]:
del df_inferring['Unnamed: 0']
df_inferring.head()

,segment_id,polygon_id,class_id,pixels_cluster,Pixels,Perimeter Pixels,alphas
0,0,0,6,"[1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...","[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0...","[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 4...","[0.34, 0.67]"
1,1,0,6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[(0, 5), (0, 6), (0, 7), (0, 8), (1, 5), (1, 6...","[(0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (4, 8...","[0.26, 0.75]"
2,2,0,6,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(0, 9), (0, 10), (0, 11), (0, 12), (1, 9), (1...","[(0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (4, 1...","[0.64, 0.37]"
3,3,0,5,"[1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, ...","[(0, 13), (0, 14), (0, 15), (0, 16), (1, 13), ...","[(0, 13), (1, 13), (2, 13), (3, 13), (4, 13), ...","[0.46, 0.55]"
4,4,0,5,"[1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, ...","[(0, 17), (0, 18), (0, 19), (1, 16), (1, 17), ...","[(0, 17), (1, 16), (2, 16), (3, 16), (4, 17), ...","[0.65, 0.36]"


In [40]:
def replace_values(row):
    return [row['alphas'][val] for val in row['pixels_cluster']]

df_inferring['heatmap'] = df_inferring.apply(replace_values, axis=1)
df_inferring.head()

,segment_id,polygon_id,class_id,pixels_cluster,Pixels,Perimeter Pixels,alphas,heatmap
0,0,0,6,"[1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, ...","[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 0...","[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 4...","[0.34, 0.67]","[0.67, 0.67, 0.67, 0.34, 0.34, 0.34, 0.34, 0.3..."
1,1,0,6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[(0, 5), (0, 6), (0, 7), (0, 8), (1, 5), (1, 6...","[(0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (4, 8...","[0.26, 0.75]","[0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.2..."
2,2,0,6,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(0, 9), (0, 10), (0, 11), (0, 12), (1, 9), (1...","[(0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (4, 1...","[0.64, 0.37]","[0.37, 0.37, 0.37, 0.37, 0.64, 0.64, 0.64, 0.6..."
3,3,0,5,"[1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, ...","[(0, 13), (0, 14), (0, 15), (0, 16), (1, 13), ...","[(0, 13), (1, 13), (2, 13), (3, 13), (4, 13), ...","[0.46, 0.55]","[0.55, 0.55, 0.46, 0.46, 0.55, 0.55, 0.46, 0.5..."
4,4,0,5,"[1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, ...","[(0, 17), (0, 18), (0, 19), (1, 16), (1, 17), ...","[(0, 17), (1, 16), (2, 16), (3, 16), (4, 17), ...","[0.65, 0.36]","[0.36, 0.36, 0.36, 0.36, 0.65, 0.65, 0.36, 0.3..."


In [41]:
# save df_inferring as csv
df_inferring.to_csv('Prepro_Data/Inference_results_20.txt', index=False)